In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

data = pd.read_excel('/Users/serdaradali/Documents/somuchmore/data/BI manager test case.xlsx')

#get rid off spaces in column names && underscore all
data.columns = [column.strip().replace(' ', '_').lower() for column in data.columns]

data_with_null = data[data.customer_id.isnull()]
data_clean = data[data.customer_id.notnull()]

# remove rows with no subscription date
data_clean_more = data_clean[data_clean.subscription_date.notnull()]

# convert customer_id from float to integer
data_clean_more.customer_id = data_clean_more.customer_id.astype(int)

grouped_customer = data_clean_more.groupby('customer_id')
customer_data = grouped_customer.agg({'subscription_date': \
                                      {'last_subsripction':np.max, 'first_subscription': np.min}})['subscription_date']
customer_data['churned'] =  customer_data['last_subsripction'].apply(lambda x: x.strftime('%Y-%m') != '2016-03')
customer_data['subscription_count'] = grouped_customer['full_product_price'].agg(['mean','count'])['count']
customer_data['avg_cost'] = grouped_customer.agg({'cost':np.mean})
customer_data['avg_revenue'] = grouped_customer.agg({'revenue':np.mean})

customer_data['joined_summer'] = customer_data['first_subscription']\
.apply(lambda x: 1 if (x.strftime('%B') == 'June') else 0)

customer_data['joined_autumn'] = customer_data['first_subscription']\
.apply(lambda x: 1 if (x.strftime('%B') == 'September' or x.strftime('%B') == 'October') else 0)

data_clean_more['usage'] = (data_clean_more['full_product_price'] - data_clean_more['revenue']) 
customer_data['avg_usage'] =  data_clean_more.groupby('customer_id').agg({'usage': 'mean' })
customer_data['country'] = grouped_customer.agg({'country':np.max}) 
customer_data['product_type'] = data_clean_more.groupby(['customer_id','product_type'],as_index=False).first()['product_type']
customer_data['marketing_channel'] = data_clean_more.groupby(['customer_id','marketing_channel'],as_index=False).first()['marketing_channel']

from datetime import datetime as dtp
customer_data['member_since'] = customer_data['first_subscription'].\
map(lambda x: (dtp.strptime('2016-03-30','%Y-%m-%d') - x).days)

customer_data['switched_product'] = data_clean_more.groupby(['customer_id'])\
.agg({'product_type': pd.Series.nunique})['product_type'].map(lambda x: x > 1)

churn_result = customer_data['churned']

customer_data['switched_product'] = np.where(customer_data['switched_product'] == True,1,0)
customer_data['subscription_day_ratio'] = customer_data['member_since'] / customer_data['subscription_count']

churn_feat_space = customer_data.drop(['churned'],axis=1)

dummy_ranks = pd.get_dummies(customer_data['product_type']).applymap(lambda x: x.astype(int))
dummy_marketing = pd.get_dummies(customer_data['marketing_channel']).applymap(lambda x: x.astype(int))
dummy_country = pd.get_dummies(customer_data['country']).applymap(lambda x: x.astype(int))

churn_feat_space = churn_feat_space.drop(['product_type'],axis=1).join(dummy_ranks.ix[:, 'product_3':])
churn_feat_space = churn_feat_space.drop(['marketing_channel'],axis=1).join(dummy_marketing.ix[:, 'Channel 1':'Channel 4'])
churn_feat_space = churn_feat_space.drop(['country'],axis=1).join(dummy_country.ix[:, 'Country 2':])

final_frame = pd.concat([churn_feat_space[['avg_cost','avg_revenue','avg_usage', 'joined_autumn', 'joined_summer', 'member_since', 'subscription_day_ratio']],\
               churn_feat_space.loc[:,'product_3':]],axis=1)

import statsmodels.api as sm

def run_Logit(X,y):
    # Construct a kfolds object
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy()
    
    # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        # Initialize a classifier with key word arguments
        clf = sm.Logit(X_train,y_train)
        result = clf.fit()
        #y_pred[test_index] = clf.predict(X_test)
    return result

target = churn_result.apply(lambda x: 1 if x else 0)

clf = sm.Logit(target,final_frame)
result = clf.fit(maxiter=1000)
print(result.summary())
print(np.exp(result.params)-1)

/Users/serdaradali/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/serdaradali/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Optimization terminated successfully.
         Current function value: 0.438078
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                churned   No. Observations:                 6900
Model:                          Logit   Df Residuals:                     6886
Method:                           MLE   Df Model:                           13
Date:                Mon, 27 Jun 2016   Pseudo R-squ.:                  0.3677
Time:                        03:30:24   Log-Likelihood:                -3022.7
converged:                       True   LL-Null:                       -4780.3
                                        LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------
avg_cost                  -0.0075      0.001     -7.012      0.000        -0.010    

Based on the logistic regression analysis results above, all features but 'product_3' and country info were found to have statistically significant impact on the customer churn. Let's interpret the results:

- Product_1 users have %29 more chance of churning than product_3 users
- Channel 1 users have %68 more chance of churning than Channel 5 users
- Channel 5 users have less chance of churning than any other marketing channel
- Users in Country 2 has %24 less chance of churning than in Country 1
- Users joined in July has %71 less chance of churning than the rest of the year